<a href="https://colab.research.google.com/github/Lambda-School-Labs/bridges-to-prosperity-ds-d/blob/main/notebooks/b2p_d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Imports
import numpy as np
import pandas as pd
from datetime import date

In [6]:
# Load data
xl_file = pd.ExcelFile('https://github.com/Lambda-School-Labs/bridges-to-prosperity-ds-d/blob/main/Data/B2P%20Dataset_2020.10.xlsx?raw=true')
dfs = {sheet_name: xl_file.parse(sheet_name) for sheet_name in xl_file.sheet_names}
print(dfs.keys())
df = dfs['Data']

dict_keys(['Data', 'Data Dictionary', 'Contextual Summary'])


In [7]:
dfs['Data Dictionary']

,Field,Description (if not self-explanatory)
0,Bridge Name,NaN
1,Bridge Opportunity: Project Code,Unique ID
2,Bridge Opportunity: Needs Assessment,"This will be populated with ""Rwanda 2018 Needs..."
3,Bridge Opportunity: Level 1 Government,Province
4,Bridge Opportunity: Level 2 Government,District
5,Bridge Opportunity: Stage,Project status of this site as of 2020.10.26
6,Bridge Opportunity: GPS (Latitude),NaN
7,Bridge Opportunity: GPS (Longitude),NaN
8,Bridge Opportunity: Bridge Type,Type of standard bridge built or recommended
9,Bridge Opportunity: Span (m),The length of the bridge as determined by seni...


In [8]:
dfs['Contextual Summary']

,Problem 1: Parsing 2013/2014 needs assessment data into current fields,"300+ sites were initially identified as part of an informal needs assessment conducted by our Rwanda field staff over 2013 and 2014. We collected much of the same data that we do now, but in a less structured format. When we created the database in 2017, we loaded all the old needs assessment into the general comments field on each bridge record (column L in this table), just to keep as a reference. We now believe that loading this data into our standard project assessments would be a benefit to future needs assessment work, both field-based and remote. We would love to see the needs assessment data from Column L parsed into the corresponding columns after it (M+), with a new row if there is existing data in those columns. If no corresponding column is obvious, we'd like to see new columns created for any extraneous data that is common to the old format."
0,Problem 2: Predicting which sites will be tech...,"Any sites with a ""Yes"" in the column AQ have u..."


In [9]:
df[['Bridge Opportunity: Stage', 'Flag for Rejection', 'Rejection Reason', 'Senior Engineering Review Conducted']][df['Senior Engineering Review Conducted'] == 'Yes']

,Bridge Opportunity: Stage,Flag for Rejection,Rejection Reason,Senior Engineering Review Conducted
0,Cancelled,Yes,There is a planned vehicular road with a vehic...,Yes
1,Complete,NaN,NaN,Yes
2,Complete,NaN,NaN,Yes
3,Complete,No,NaN,Yes
4,Complete,No,NaN,Yes
...,...,...,...,...
1105,Rejected,No,NaN,Yes
1196,Rejected,No,NaN,Yes
1197,Rejected,No,NaN,Yes
1200,Rejected,No,NaN,Yes


In [10]:
df.sample(10, random_state=42, axis=0)

,Bridge Name,Bridge Opportunity: Project Code,Bridge Opportunity: Needs Assessment,Bridge Opportunity: Level 1 Government,Bridge Opportunity: Level 2 Government,Bridge Opportunity: Stage,Bridge Opportunity: GPS (Latitude),Bridge Opportunity: GPS (Longitude),Bridge Opportunity: Bridge Type,Bridge Opportunity: Span (m),Bridge Opportunity: Individuals Directly Served,Bridge Opportunity: Comments,Form: Form Name,Form: Created By,Proposed Bridge Location (GPS) (Latitude),Proposed Bridge Location (GPS) (Longitude),Current crossing method,Nearest all-weather crossing point,Days per year river is flooded,Flood duration during rainy season,Market access blocked by river,Education access blocked by river,Health access blocked by river,Other access blocked by river,Primary occupations,Primary crops grown,River crossing deaths in last 3 years,River crossing injuries in last 3 years,Incident descriptions,Notes on social information,Cell service quality,4WD Accessibility,Name of nearest city,Name of nearest paved or sealed road,Bridge classification,Flag for Rejection,Rejection Reason,Bridge Type,Estimated span (m),Height differential between banks,Bridge Opportunity: General Project Photos,Bridge Opportunity: CaseSafeID,Senior Engineering Review Conducted
852,Ruhengeri 2,1013633,Rwanda Needs Assessment 2018,Western Province,Rutsiro,Rejected,-1.880270,29.342770,NaN,NaN,519.0,NaN,Project Assessment - 2018.9.27,cesarngenzi taroworks,-1.880270,29.342770,Timber bridge,I have assessed another pedestrian bridge in 4...,28.0,30 minutes,Gakeri,None,None,None,Farming,Potatoes,2.0,1.0,A child has fell in the river but fortunately ...,The community of ruhengeri village cross to vi...,Good,Never,Gakeri,Rubavu-Rubengera,Standard,No,NaN,Suspended,50.0,NaN,NaN,006f100000d7iRnAAI,NaN
184,Kabira1,1013646,Rwanda Needs Assessment 2018,Western Province,Rutsiro,Identified,-1.977500,29.381944,NaN,NaN,2794.0,NaN,Project Assessment - 2018.9.27,cesarngenzi taroworks,-1.977500,29.381944,None,None,28.0,Thirty minutes,Congo Nile,Groupe scolaire de la paix,Congo Nile health center,ADPR in mucyebera,Farming,Cassava and beans,0.0,3.0,Olde women fell in coming from their farms,"Market(gishushu, kabiraho,), Sec school (gishu...",Good,Never,Congo Nile,Rubavu-rubengera,Standard,No,NaN,Suspended,53.0,NaN,NaN,006f100000d7iVzAAI,NaN
1261,Kinyinya,1014347,Rwanda Needs Assessment 2018,Western Province,Karongi,Rejected,-2.072450,29.604690,NaN,NaN,3105.0,NaN,Project Assessment - 2018.11.18,edouardumwanzuro taroworks,-2.072450,29.604690,None,None,98.0,2 hours of flood,"Mwunguzi village, Kirehe village, nzobe villag...","Mwunguzi village, Kirehe village, nzobe villag...","Mwunguzi village, Kirehe village, nzobe villag...","Mwunguzi village, Kirehe village, nzobe villag...",Farming,"Potatoes, beans",0.0,1.0,One injury across this river,mwunguzi broadcast officer provided info about...,Fair,Never,Karongi,Karongi - Muhanga,Standard,No,NaN,Suspension,33.0,NaN,NaN,006f100000eeuuOAAQ,NaN
67,Muduha,1007491,Rwanda Needs Assessment 2018,Southern Province,Nyamagabe,Identified,-2.436162,29.523080,NaN,17.0,4530.0,"1000-3000 people directly served, Elevation:18...",Project Assessment - 2018.9.1,aimablengirabakunzi taroworks,-2.436162,29.523080,Timber,Vehicular bridge in 1km away from the proposed...,60.0,All the Times in April and May,Market of Musebeya and Nyamagabe,Primary school,No Health Center in this location,Church,"Farming of sweet potatoes, beans, coffee, forest","Coffee, beans, sweet potatoes, trees",5.0,0.0,They slipped on trees and fall down in the riv...,"Market Musebeya(Muduha, Gasharu, Rukamiro, Nya...",Nonexistent,Seasonally,Huye,Nyamagabe-Rusizi,Standard,No,NaN,Suspended,75.4,1.0,NaN,006f100000a86FTAAY,NaN
220,Kabuga,1014038,Rwanda Needs Assessment 2018,Western Province,Karongi,Identified,-2.246215,29.491565,NaN,NaN,21209.0,NaN,Project Assessment - 2018.10.28,aimablengirabakunzi taroworks,-2.246215,29.491565,None,Vehicular bridge in 2.5km downstream of the pr...,140.0,20times thr

In [11]:
df.isnull().sum()

Bridge Name                                           0
Bridge Opportunity: Project Code                      0
Bridge Opportunity: Needs Assessment                 40
Bridge Opportunity: Level 1 Government                0
Bridge Opportunity: Level 2 Government                0
Bridge Opportunity: Stage                             0
Bridge Opportunity: GPS (Latitude)                   12
Bridge Opportunity: GPS (Longitude)                  12
Bridge Opportunity: Bridge Type                    1287
Bridge Opportunity: Span (m)                       1082
Bridge Opportunity: Individuals Directly Served       6
Bridge Opportunity: Comments                       1078
Form: Form Name                                       0
Form: Created By                                      0
Proposed Bridge Location (GPS) (Latitude)            40
Proposed Bridge Location (GPS) (Longitude)           40
Current crossing method                             160
Nearest all-weather crossing point              

In [12]:
# columns to drop
cols = ['Form: Created By', 'Incident descriptions', 'Notes on social information', 
        'Bridge Opportunity: Comments', 'Bridge Opportunity: General Project Photos']
df = df.drop(columns=cols)

In [13]:
print(df.shape)
df.dtypes

(1472, 38)


Bridge Name                                         object
Bridge Opportunity: Project Code                    object
Bridge Opportunity: Needs Assessment                object
Bridge Opportunity: Level 1 Government              object
Bridge Opportunity: Level 2 Government              object
Bridge Opportunity: Stage                           object
Bridge Opportunity: GPS (Latitude)                 float64
Bridge Opportunity: GPS (Longitude)                float64
Bridge Opportunity: Bridge Type                     object
Bridge Opportunity: Span (m)                       float64
Bridge Opportunity: Individuals Directly Served    float64
Form: Form Name                                     object
Proposed Bridge Location (GPS) (Latitude)          float64
Proposed Bridge Location (GPS) (Longitude)         float64
Current crossing method                             object
Nearest all-weather crossing point                  object
Days per year river is flooded                     float

In [14]:
df.iloc[100:101,:]

,Bridge Name,Bridge Opportunity: Project Code,Bridge Opportunity: Needs Assessment,Bridge Opportunity: Level 1 Government,Bridge Opportunity: Level 2 Government,Bridge Opportunity: Stage,Bridge Opportunity: GPS (Latitude),Bridge Opportunity: GPS (Longitude),Bridge Opportunity: Bridge Type,Bridge Opportunity: Span (m),Bridge Opportunity: Individuals Directly Served,Form: Form Name,Proposed Bridge Location (GPS) (Latitude),Proposed Bridge Location (GPS) (Longitude),Current crossing method,Nearest all-weather crossing point,Days per year river is flooded,Flood duration during rainy season,Market access blocked by river,Education access blocked by river,Health access blocked by river,Other access blocked by river,Primary occupations,Primary crops grown,River crossing deaths in last 3 years,River crossing injuries in last 3 years,Cell service quality,4WD Accessibility,Name of nearest city,Name of nearest paved or sealed road,Bridge classification,Flag for Rejection,Rejection Reason,Bridge Type,Estimated span (m),Height differential between banks,Bridge Opportunity: CaseSafeID,Senior Engineering Review Conducted
100,Gatembe,1007898,Rwanda Needs Assessment 2018,Northern Province,Gakenke,Identified,-1.65575,29.8198,NaN,48.0,8000.0,Project Assessment - 2018.12.12,-1.65575,29.8198,None,None,365.0,NaN,Gakenke,Kirebe high school,Nemba hospital,None,Farming,Maize,4.0,3.0,Good,Year Round,Gakenke,Kigali-musanze,Standard,No,NaN,Suspended,55.0,NaN,006f100000c5VbqAAE,NaN


In [15]:
df[df['Bridge Opportunity: Project Code'].str.match('[a-zA-Z]') == True]
# df['Bridge Opportunity: Project Code'] = df['Bridge Opportunity: Project Code'].astype(int)

,Bridge Name,Bridge Opportunity: Project Code,Bridge Opportunity: Needs Assessment,Bridge Opportunity: Level 1 Government,Bridge Opportunity: Level 2 Government,Bridge Opportunity: Stage,Bridge Opportunity: GPS (Latitude),Bridge Opportunity: GPS (Longitude),Bridge Opportunity: Bridge Type,Bridge Opportunity: Span (m),Bridge Opportunity: Individuals Directly Served,Form: Form Name,Proposed Bridge Location (GPS) (Latitude),Proposed Bridge Location (GPS) (Longitude),Current crossing method,Nearest all-weather crossing point,Days per year river is flooded,Flood duration during rainy season,Market access blocked by river,Education access blocked by river,Health access blocked by river,Other access blocked by river,Primary occupations,Primary crops grown,River crossing deaths in last 3 years,River crossing injuries in last 3 years,Cell service quality,4WD Accessibility,Name of nearest city,Name of nearest paved or sealed road,Bridge classification,Flag for Rejection,Rejection Reason,Bridge Type,Estimated span (m),Height differential between banks,Bridge Opportunity: CaseSafeID,Senior Engineering Review Conducted
42,Gasura,RW104,NaN,Kigali,Gasabo,Complete,-1.820833,30.113889,Suspension Bridge,44.0,7000.0,Inspection Form - Suspension - 2018.2.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,006f100000ZQRq3AAH,Yes
43,Kucyaruseke,RW127,NaN,Southern Province,Nyaruguru,Complete,-2.711764,29.557267,Suspended Bridge,79.0,3000.0,Project Assessment - 2017.9.5,-2.711764,29.557267,Log timber bridge,23km upstream,90.0,The river can flood 24 hours. No one crossing ...,On both sides there are markets and when the r...,NaN,NaN,NaN,"Farming, teaching, masons","Beans, tea, potatoes",0.0,0.0,Fair,NaN,Huye,Butare-Burundi (NR19),NaN,NaN,NaN,NaN,6.0,0.3,006f100000Zs7KqAAJ,Yes


In [28]:
dfw = pd.read_excel("https://github.com/Lambda-School-Labs/bridges-to-prosperity-ds-d/blob/main/Data/Bridges%20Dataset_2020.01.14.xls?raw=true")

In [29]:
dfw.head()

,Country,Level 1 Government,Level 2 Government,Bridge Name,Project Code,B2P Fiscal Year,Stage,Individuals Directly Served,GPS (Latitude),GPS (Longitude),Bridge Type,Span (m),Project Type
0,Uganda,Eastern Region,Mbale,Kama,UG101,2018,Complete,5280.0,1.116330,34.252460,Suspended Bridge,90.0,B2P Built
1,Swaziland,NaN,NaN,Mbluzi-Myendle,GO108,2017,Complete,4000.0,-26.294120,31.271240,Suspended Bridge,73.0,B2P Supported
2,Panama,Ngäbe-Buglé,NaN,Llano Bonito,PA122,2017,Complete,1500.0,8.446314,-81.365675,Suspended Bridge,79.0,B2P Supported
3,Nicaragua,Jinotega,Santa María de Pantasma,Puente San Vicente N°1,1008677,2018,Complete,4000.0,13.435012,-85.783800,Suspension Bridge,67.8,B2P Built
4,Nicaragua,Matagalpa,Río Blanco,La Guitarrilla,1008691,2018,Complete,4000.0,13.122785,-85.236833,Suspension Bridge,77.5,B2P Built


In [30]:
df.columns

Index(['Bridge Name', 'Bridge Opportunity: Project Code',
       'Bridge Opportunity: Needs Assessment',
       'Bridge Opportunity: Level 1 Government',
       'Bridge Opportunity: Level 2 Government', 'Bridge Opportunity: Stage',
       'Bridge Opportunity: GPS (Latitude)',
       'Bridge Opportunity: GPS (Longitude)',
       'Bridge Opportunity: Bridge Type', 'Bridge Opportunity: Span (m)',
       'Bridge Opportunity: Individuals Directly Served', 'Form: Form Name',
       'Proposed Bridge Location (GPS) (Latitude)',
       'Proposed Bridge Location (GPS) (Longitude)', 'Current crossing method',
       'Nearest all-weather crossing point', 'Days per year river is flooded',
       'Flood duration during rainy season', 'Market access blocked by river',
       'Education access blocked by river', 'Health access blocked by river',
       'Other access blocked by river', 'Primary occupations',
       'Primary crops grown', 'River crossing deaths in last 3 years',
       'River crossing i

In [31]:
dfw.columns

Index(['Country', 'Level 1 Government', 'Level 2 Government', 'Bridge Name',
       'Project Code', 'B2P Fiscal Year', 'Stage',
       'Individuals Directly Served', 'GPS (Latitude)', 'GPS (Longitude)',
       'Bridge Type', 'Span (m)', 'Project Type'],
      dtype='object')

In [32]:
dfw = dfw.rename(columns={'Project Code': 'Bridge Opportunity: Project Code', 
                         'Level 1 Government': 'Bridge Opportunity: Level 1 Government',
                         'Level 2 Government': 'Bridge Opportunity: Level 2 Government',
                         'Project Code': 'Bridge Opportunity: Project Code',
                         'Stage': 'Bridge Opportunity: Stage',
                         'Individuals Directly Served': 'Bridge Opportunity: Individuals Directly Served',
                         'GPS (Latitude)': 'Bridge Opportunity: GPS (Latitude)',
                         'GPS (Longitude)': 'Bridge Opportunity: GPS (Longitude)',
                         'Bridge Type': 'Bridge Opportunity: Bridge Type',
                         'Span (m)': 'Bridge Opportunity: Span (m)'})

In [33]:
dfw.head()

,Country,Bridge Opportunity: Level 1 Government,Bridge Opportunity: Level 2 Government,Bridge Name,Bridge Opportunity: Project Code,B2P Fiscal Year,Bridge Opportunity: Stage,Bridge Opportunity: Individuals Directly Served,Bridge Opportunity: GPS (Latitude),Bridge Opportunity: GPS (Longitude),Bridge Opportunity: Bridge Type,Bridge Opportunity: Span (m),Project Type
0,Uganda,Eastern Region,Mbale,Kama,UG101,2018,Complete,5280.0,1.116330,34.252460,Suspended Bridge,90.0,B2P Built
1,Swaziland,NaN,NaN,Mbluzi-Myendle,GO108,2017,Complete,4000.0,-26.294120,31.271240,Suspended Bridge,73.0,B2P Supported
2,Panama,Ngäbe-Buglé,NaN,Llano Bonito,PA122,2017,Complete,1500.0,8.446314,-81.365675,Suspended Bridge,79.0,B2P Supported
3,Nicaragua,Jinotega,Santa María de Pantasma,Puente San Vicente N°1,1008677,2018,Complete,4000.0,13.435012,-85.783800,Suspension Bridge,67.8,B2P Built
4,Nicaragua,Matagalpa,Río Blanco,La Guitarrilla,1008691,2018,Complete,4000.0,13.122785,-85.236833,Suspension Bridge,77.5,B2P Built


In [36]:
for col in dfw.columns:
  if col not in df.columns:
    print(col)
  else:
    print("\t", col, "exists in df")

Country
	 Bridge Opportunity: Level 1 Government exists in df
	 Bridge Opportunity: Level 2 Government exists in df
	 Bridge Name exists in df
	 Bridge Opportunity: Project Code exists in df
B2P Fiscal Year
	 Bridge Opportunity: Stage exists in df
	 Bridge Opportunity: Individuals Directly Served exists in df
	 Bridge Opportunity: GPS (Latitude) exists in df
	 Bridge Opportunity: GPS (Longitude) exists in df
	 Bridge Opportunity: Bridge Type exists in df
	 Bridge Opportunity: Span (m) exists in df
Project Type


In [37]:
dfw['Country'].value_counts()

Rwanda           68
Nicaragua        60
Bolivia          57
Panama           29
Guatemala        25
Uganda           18
Ethiopia         15
Haiti            12
Honduras         11
Peru             10
Zambia            8
El Salvador       7
Timor-Leste       5
Swaziland         3
Nepal             2
Kenya             2
Philippines       2
United States     1
Indonesia         1
Sudan             1
Mozambique        1
Colombia          1
Name: Country, dtype: int64

In [38]:
df["Country"] = "Rwanda"
df.head()

,Bridge Name,Bridge Opportunity: Project Code,Bridge Opportunity: Needs Assessment,Bridge Opportunity: Level 1 Government,Bridge Opportunity: Level 2 Government,Bridge Opportunity: Stage,Bridge Opportunity: GPS (Latitude),Bridge Opportunity: GPS (Longitude),Bridge Opportunity: Bridge Type,Bridge Opportunity: Span (m),Bridge Opportunity: Individuals Directly Served,Form: Form Name,Proposed Bridge Location (GPS) (Latitude),Proposed Bridge Location (GPS) (Longitude),Current crossing method,Nearest all-weather crossing point,Days per year river is flooded,Flood duration during rainy season,Market access blocked by river,Education access blocked by river,Health access blocked by river,Other access blocked by river,Primary occupations,Primary crops grown,River crossing deaths in last 3 years,River crossing injuries in last 3 years,Cell service quality,4WD Accessibility,Name of nearest city,Name of nearest paved or sealed road,Bridge classification,Flag for Rejection,Rejection Reason,Bridge Type,Estimated span (m),Height differential between banks,Bridge Opportunity: CaseSafeID,Senior Engineering Review Conducted,Country
0,Bukinga,1009317,Rwanda Needs Assessment 2018,Southern Province,Nyaruguru,Cancelled,-2.760833,29.488056,Suspended Bridge,NaN,NaN,Project Assessment - 2018.10.10,-2.760833,29.488056,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Huye,NaN,NaN,Yes,There is a planned vehicular road with a vehic...,NaN,NaN,NaN,006f100000asFErAAM,Yes,Rwanda
1,Kagarama,1007325,NaN,Western Province,Nyamasheke,Complete,-2.322534,29.141944,Suspension Bridge,48.0,4000.0,Project Assessment - 2017.7.12,-2.322534,29.141944,timber bridge,2km downstream,120.0,12hours,"Schools, hospital, markets and government serv...",NaN,NaN,NaN,farming,banana,2.0,15.0,Fair,Seasonally,Kibogora-Kabuga,Karongi-Nyamasheke,NaN,NaN,NaN,NaN,10.0,0.7,006f100000a82QzAAI,Yes,Rwanda
2,Karambi,1007374,NaN,Southern Province,Huye,Complete,-2.513333,29.612778,Suspension Bridge,50.0,12500.0,Project Assessment - 2018.7.27,NaN,NaN,Timber bridge,Is located at 5km upstream. People pay 1000rwf...,60.0,1day,"Markets,farms blocked by this river both sides",Education blocked by this river both sides,Health services blocked by this river both sides,"Potable water, places of worship ,places for r...",Farming and small business,"Corns, banana and beans",3.0,15.0,Good,Never,Nyamagabe,Huye-Nyamagabe,Standard,NaN,NaN,Suspension,45.0,1.0,006f100000a86DaAAI,Yes,Rwanda
3,Rugeti,1007466,Rwanda Needs Assessment 2018,Southern Province,Nyamagabe,Complete,-2.476407,29.420800,Suspension Bridge,68.0,4363.0,Project Assessment - 2018.9.1,-2.476407,29.420800,Timber,None,60.0,All time in April and May and it is rarely to ...,Markets,Primary and Secondary schools,Health Center,Churches and Government offices,"Farming of large tea plantation, wheat farms","Tea, cow pea, wheat, beans, sweet potatoes",0.0,2.0,Fair,Seasonally,Huye,Nyamagabe-Rusizi,Standard,No,NaN,Suspended,47.4,1.1,006f100000a86F4AAI,Yes,Rwanda
4,Nyakabuye - Nkomane,1007495,Rwanda Needs Assessment 2018,Southern Province,Nyamagabe,Complete,-2.233022,29.575130,Suspension Bridge,78.0,2790.0,Project Assessment - 2018.9.4,-2.233574,29.575366,Timber bridge,"Vehicular bridge, 5 kilometers",35.0,"April To May, September to December, 12 hours ...","Birambo market, Gasura market, kirinda market(...","Groupe scholar Gasave( 1,2)","Kirinda Hospital(villages 1,2)",None,Farming,"Cassava, maize, beans, sorghum, sweet potatoes",4.0,4.0,Good,Never,Nyanza,Nyamagabe,Standard,No,NaN,Suspended,90.0,1.1,006f100000a86FXAAY,Yes,Rwanda


In [45]:
dfw.shape, df.shape

((339, 13), (1472, 39))

In [44]:
print(dfw["Bridge Opportunity: Project Code"].isna().sum())
df["Bridge Opportunity: Project Code"].apply(lambda x: x in dfw["Bridge Opportunity: Project Code"]).sum()

0


0

In [49]:
df_new = pd.concat([df, dfw])
print(df_new.shape)
df_new.head()

(1811, 41)


,Bridge Name,Bridge Opportunity: Project Code,Bridge Opportunity: Needs Assessment,Bridge Opportunity: Level 1 Government,Bridge Opportunity: Level 2 Government,Bridge Opportunity: Stage,Bridge Opportunity: GPS (Latitude),Bridge Opportunity: GPS (Longitude),Bridge Opportunity: Bridge Type,Bridge Opportunity: Span (m),Bridge Opportunity: Individuals Directly Served,Form: Form Name,Proposed Bridge Location (GPS) (Latitude),Proposed Bridge Location (GPS) (Longitude),Current crossing method,Nearest all-weather crossing point,Days per year river is flooded,Flood duration during rainy season,Market access blocked by river,Education access blocked by river,Health access blocked by river,Other access blocked by river,Primary occupations,Primary crops grown,River crossing deaths in last 3 years,River crossing injuries in last 3 years,Cell service quality,4WD Accessibility,Name of nearest city,Name of nearest paved or sealed road,Bridge classification,Flag for Rejection,Rejection Reason,Bridge Type,Estimated span (m),Height differential between banks,Bridge Opportunity: CaseSafeID,Senior Engineering Review Conducted,Country,B2P Fiscal Year,Project Type
0,Bukinga,1009317,Rwanda Needs Assessment 2018,Southern Province,Nyaruguru,Cancelled,-2.760833,29.488056,Suspended Bridge,NaN,NaN,Project Assessment - 2018.10.10,-2.760833,29.488056,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Huye,NaN,NaN,Yes,There is a planned vehicular road with a vehic...,NaN,NaN,NaN,006f100000asFErAAM,Yes,Rwanda,NaN,NaN
1,Kagarama,1007325,NaN,Western Province,Nyamasheke,Complete,-2.322534,29.141944,Suspension Bridge,48.0,4000.0,Project Assessment - 2017.7.12,-2.322534,29.141944,timber bridge,2km downstream,120.0,12hours,"Schools, hospital, markets and government serv...",NaN,NaN,NaN,farming,banana,2.0,15.0,Fair,Seasonally,Kibogora-Kabuga,Karongi-Nyamasheke,NaN,NaN,NaN,NaN,10.0,0.7,006f100000a82QzAAI,Yes,Rwanda,NaN,NaN
2,Karambi,1007374,NaN,Southern Province,Huye,Complete,-2.513333,29.612778,Suspension Bridge,50.0,12500.0,Project Assessment - 2018.7.27,NaN,NaN,Timber bridge,Is located at 5km upstream. People pay 1000rwf...,60.0,1day,"Markets,farms blocked by this river both sides",Education blocked by this river both sides,Health services blocked by this river both sides,"Potable water, places of worship ,places for r...",Farming and small business,"Corns, banana and beans",3.0,15.0,Good,Never,Nyamagabe,Huye-Nyamagabe,Standard,NaN,NaN,Suspension,45.0,1.0,006f100000a86DaAAI,Yes,Rwanda,NaN,NaN
3,Rugeti,1007466,Rwanda Needs Assessment 2018,Southern Province,Nyamagabe,Complete,-2.476407,29.420800,Suspension Bridge,68.0,4363.0,Project Assessment - 2018.9.1,-2.476407,29.420800,Timber,None,60.0,All time in April and May and it is rarely to ...,Markets,Primary and Secondary schools,Health Center,Churches and Government offices,"Farming of large tea plantation, wheat farms","Tea, cow pea, wheat, beans, sweet potatoes",0.0,2.0,Fair,Seasonally,Huye,Nyamagabe-Rusizi,Standard,No,NaN,Suspended,47.4,1.1,006f100000a86F4AAI,Yes,Rwanda,NaN,NaN
4,Nyakabuye - Nkomane,1007495,Rwanda Needs Assessment 2018,Southern Province,Nyamagabe,Complete,-2.233022,29.575130,Suspension Bridge,78.0,2790.0,Project Assessment - 2018.9.4,-2.233574,29.575366,Timber bridge,"Vehicular bridge, 5 kilometers",35.0,"April To May, September to December, 12 hours ...","Birambo market, Gasura market, kirinda market(...","Groupe scholar Gasave( 1,2)","Kirinda Hospital(villages 1,2)",None,Farming,"Cassava, maize, beans, sorghum, sweet potatoes",4.0,4.0,Good,Never,Nyanza,Nyamagabe,Standard,No,NaN,Suspended,90.0,1.1,006f100000a86FXAAY,Yes,Rwanda,NaN,NaN


In [48]:
df_new["Bridge Opportunity: Stage"].value_counts()

Rejected              1175
Complete               356
Identified             224
Under Construction      26
Prospecting             18
Confirmed               11
Cancelled                1
Name: Bridge Opportunity: Stage, dtype: int64

In [51]:
df[df['Bridge Opportunity: Project Code'] == '1007332']

,Bridge Name,Bridge Opportunity: Project Code,Bridge Opportunity: Needs Assessment,Bridge Opportunity: Level 1 Government,Bridge Opportunity: Level 2 Government,Bridge Opportunity: Stage,Bridge Opportunity: GPS (Latitude),Bridge Opportunity: GPS (Longitude),Bridge Opportunity: Bridge Type,Bridge Opportunity: Span (m),Bridge Opportunity: Individuals Directly Served,Form: Form Name,Proposed Bridge Location (GPS) (Latitude),Proposed Bridge Location (GPS) (Longitude),Current crossing method,Nearest all-weather crossing point,Days per year river is flooded,Flood duration during rainy season,Market access blocked by river,Education access blocked by river,Health access blocked by river,Other access blocked by river,Primary occupations,Primary crops grown,River crossing deaths in last 3 years,River crossing injuries in last 3 years,Cell service quality,4WD Accessibility,Name of nearest city,Name of nearest paved or sealed road,Bridge classification,Flag for Rejection,Rejection Reason,Bridge Type,Estimated span (m),Height differential between banks,Bridge Opportunity: CaseSafeID,Senior Engineering Review Conducted,Country
295,Bugambo,1007332,Rwanda Needs Assessment 2018,Northern Province,Burera,Rejected,-1.377,29.80729,NaN,29.0,1892.0,Project Assessment - 2018.9.16,-1.377,29.80729,Timber,Vehicular bridge in two kilometers away,196.0,7 days,There is a part of gatare which is blocked to ...,There is a part of gatare which is blocked to ...,There is a part of gatare which is blocked to ...,There is a part of gatare which is blocked to ...,"School, Market, bank, farms",Potatoes,3.0,4.0,Good,Never,Musanze,Musanze Cyanika,Standard,No,NaN,Suspension,34.0,NaN,006f100000a86CuAAI,NaN,Rwanda
